In [10]:
import requests
from bs4 import BeautifulSoup
import json
import time
import uuid

# Функция для получения данных с одной страницы
def get_page_data(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    data = []
 
 # Извлечение данных с каждой страницы
    for item in soup.find_all('div', id='__nuxt'):
        link = item.find('a')['href']
        title = item.select('.catalog__item--vertical .catalog__title')
        price = item.select('.catalog__price')
        mileage =  item.select('.catalog__tech-list>:nth-child(3)')
        year = item.select('.catalog__tech-list>:nth-child(2)')
        power = item.select('.catalog__tech-list>:first-child')

        data.append({
            'id': uuid.uuid4(),
            'title': title,
            'price': price,
            'mileage': mileage,
            'year': year,
            'power': power
            # ... добавить другие поля
        })
        return data
# Функция для итерации по страницам
def iterate_pages(url, page_num):
    json_data = []
    for page in range(page_num):
        page_url = f"{url}?page={page + 1}"
        page_data = get_page_data(page_url)
        json_data.append(page_data)
    return json_data
time.sleep(3)

# Пример использования
url = 'https://carro.ru/used/'
page_num = 5 # Количество страниц для итерации

# Итерация по страницам и запись данных в JSON
json_data = iterate_pages(url, page_num)

In [11]:
import pandas as pd
import re

In [12]:
flattened_data = [item[0] for item in json_data]

In [179]:
df = pd.DataFrame(json_data)

In [13]:
titles = [re.sub('<[^<]+?>', '', str(item)) for item in flattened_data[0]['title']]
powers = [int(re.findall(r'(\d+)', str(item))[0]) for item in flattened_data[0]['power']]

In [14]:
df = pd.DataFrame({'Title': titles, 'Power (hp)': powers})

In [15]:
df.shape

(18, 2)

In [110]:
# Извлекаем год с помощью регулярного выражения
df['year'] = df['price'].str.extract(r'(\d{4})')

#Преобразуем в число
df['year'] = pd.to_numeric(df['year'])

In [111]:
#Обработка мощности: извлекаем число из строки
df['power'] = df['price'].str.extract('(\d+)').astype(int)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_2819/3466029246.py:1: SyntaxWarning: invalid escape sequence '\d'
  df['power'] = df['price'].str.extract('(\d+)').astype(int)


In [113]:
#Удаляем лишний столбец 'price' - данные уже обработаны и помещены в новые колонки.
df = df.drop(columns=['price'])

In [108]:
#Обработка пробега: извлекаем число из строки
def extract_price(text):
    try:
        return text.split('л.с. ')[-1][4:][:-4]
    except (IndexError, AttributeError):
        return None # или какое-то другое значение по умолчанию, например, 0

# Пример использования:
df['mileage'] = df['price'].astype(str).apply(extract_price)
df['mileage'] = df['mileage'].astype(str).apply(lambda x: int(re.sub(r'[^\d]', '', x)) if re.sub(r'[^\d]', '', x) else None)

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

def parse_carro_ru(base_url, output_filename="carro_data.json"):
    """
    Парсит страницы carro.ru с пагинацией и сохраняет данные в JSON.

    Args:
        base_url: Базовый URL сайта (например, "https://www.carro.ru/cars/").
        output_filename: Имя выходного JSON файла.
    """

    all_cars = []
    page_num = 1
    while True:
        url = f"{base_url}?page={page_num}"
        print(f"Parsing page: {url}")

        try:
            response = requests.get(url)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, "html.parser")

            #  !!! ВАЖНО !!!  Замените эти селекторы на корректные после инспекции HTML carro.ru
            car_elements = soup.find_all('div', id='__nuxt') #  Пример!  Найдите правильный класс!
            if not car_elements:  #Если элементов нет - значит страниц больше нет
                break


            for car_element in car_elements:
                try:
                    title = car_element.select('.catalog__item--vertical .catalog__title')
                    price = car_element.select('.catalog__price')
                    mileage =  car_element.select('.catalog__tech-list>:nth-child(3)')
                    year = car_element.select('.catalog__tech-list>:nth-child(2)')
                    power = car_element.select('.catalog__tech-list>:first-child')
                    # ... другие данные ... (Добавьте извлечение других необходимых данных,  например, год, пробег, марка и модель)
                    car_data = {
                    'title': title,
                    'price': price,
                    'mileage': mileage,
                    'year': year,
                    'power': power
                        # ... другие поля ...
                    }
                    all_cars.append(car_data)
                except AttributeError:
                    print("Error parsing car data. Skipping.")


            # !!! ВАЖНО !!!  Найдите селектор для элемента пагинации, указывающего на следующую страницу.
            # Это может быть ссылка, кнопка или другой элемент.
            next_page_element = car_element.select('.p-paginator-element:last-child') # Пример!  Найдите правильный селектор!
            if next_page_element is None:
                break
            #page_num = int(next_page_element['href'].split('=')[-1]) #Для случая если номер страницы в URL
            page_num += 1 #Для случая если пагинация работает через увеличение номера страницы


        except requests.exceptions.RequestException as e:
            print(f"Error fetching page: {e}")
            break

    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(all_cars, f, ensure_ascii=False, indent=4)

    print(f"Data saved to {output_filename}")


# Пример использования:
base_url = "https://www.carro.ru/used/"  # Замените на корректный URL, если нужно
parse_carro_ru(base_url)

Parsing page: https://www.carro.ru/used/?page=1
Parsing page: https://www.carro.ru/used/?page=2
Parsing page: https://www.carro.ru/used/?page=3
Parsing page: https://www.carro.ru/used/?page=4
Parsing page: https://www.carro.ru/used/?page=5
Parsing page: https://www.carro.ru/used/?page=6
Parsing page: https://www.carro.ru/used/?page=7
Parsing page: https://www.carro.ru/used/?page=8
Parsing page: https://www.carro.ru/used/?page=9
Parsing page: https://www.carro.ru/used/?page=10
Parsing page: https://www.carro.ru/used/?page=11
Parsing page: https://www.carro.ru/used/?page=12
Parsing page: https://www.carro.ru/used/?page=13
Parsing page: https://www.carro.ru/used/?page=14
Parsing page: https://www.carro.ru/used/?page=15
Parsing page: https://www.carro.ru/used/?page=16
Parsing page: https://www.carro.ru/used/?page=17
Parsing page: https://www.carro.ru/used/?page=18
Parsing page: https://www.carro.ru/used/?page=19
Parsing page: https://www.carro.ru/used/?page=20
Parsing page: https://www.car

Parsing page: https://www.carro.ru/used/?page=168
Parsing page: https://www.carro.ru/used/?page=169
Parsing page: https://www.carro.ru/used/?page=170
Parsing page: https://www.carro.ru/used/?page=171
Parsing page: https://www.carro.ru/used/?page=172
Parsing page: https://www.carro.ru/used/?page=173
Parsing page: https://www.carro.ru/used/?page=174
Parsing page: https://www.carro.ru/used/?page=175
Parsing page: https://www.carro.ru/used/?page=176
Parsing page: https://www.carro.ru/used/?page=177
Parsing page: https://www.carro.ru/used/?page=178
Parsing page: https://www.carro.ru/used/?page=179
Parsing page: https://www.carro.ru/used/?page=180
Parsing page: https://www.carro.ru/used/?page=181
Parsing page: https://www.carro.ru/used/?page=182
Parsing page: https://www.carro.ru/used/?page=183
Parsing page: https://www.carro.ru/used/?page=184
Parsing page: https://www.carro.ru/used/?page=185
Parsing page: https://www.carro.ru/used/?page=186
Parsing page: https://www.carro.ru/used/?page=187


Parsing page: https://www.carro.ru/used/?page=332
Parsing page: https://www.carro.ru/used/?page=333
Parsing page: https://www.carro.ru/used/?page=334
Parsing page: https://www.carro.ru/used/?page=335
Parsing page: https://www.carro.ru/used/?page=336
Parsing page: https://www.carro.ru/used/?page=337
Parsing page: https://www.carro.ru/used/?page=338
Parsing page: https://www.carro.ru/used/?page=339
Parsing page: https://www.carro.ru/used/?page=340
Parsing page: https://www.carro.ru/used/?page=341
Parsing page: https://www.carro.ru/used/?page=342
Parsing page: https://www.carro.ru/used/?page=343
Parsing page: https://www.carro.ru/used/?page=344
Parsing page: https://www.carro.ru/used/?page=345
Parsing page: https://www.carro.ru/used/?page=346
Parsing page: https://www.carro.ru/used/?page=347
Parsing page: https://www.carro.ru/used/?page=348
Parsing page: https://www.carro.ru/used/?page=349
Parsing page: https://www.carro.ru/used/?page=350
Parsing page: https://www.carro.ru/used/?page=351


Parsing page: https://www.carro.ru/used/?page=496
Parsing page: https://www.carro.ru/used/?page=497
Parsing page: https://www.carro.ru/used/?page=498
Parsing page: https://www.carro.ru/used/?page=499
Parsing page: https://www.carro.ru/used/?page=500
Parsing page: https://www.carro.ru/used/?page=501
Parsing page: https://www.carro.ru/used/?page=502
Parsing page: https://www.carro.ru/used/?page=503
Parsing page: https://www.carro.ru/used/?page=504
Parsing page: https://www.carro.ru/used/?page=505
Parsing page: https://www.carro.ru/used/?page=506
Parsing page: https://www.carro.ru/used/?page=507
Parsing page: https://www.carro.ru/used/?page=508
Parsing page: https://www.carro.ru/used/?page=509
Parsing page: https://www.carro.ru/used/?page=510
Parsing page: https://www.carro.ru/used/?page=511
Parsing page: https://www.carro.ru/used/?page=512
Parsing page: https://www.carro.ru/used/?page=513
Parsing page: https://www.carro.ru/used/?page=514
Parsing page: https://www.carro.ru/used/?page=515


In [258]:
url = '/home/max/carro_data.json'

In [260]:
df = pd.read_json(url)

ValueError: Expected object or value